# Título: [Não sei]

## Este projeto tem o objetivo de consolidar os conhecimentos vistos até o momento envolvendo as bibliotecas básicas de análise de dados: Numpy, Pandas e Matplotlib.

Para isso, serão utilizadas as bases de dados MovieLens:

* **Data Source:** MovieLens web site (filename: ml-25m.zip)
* **Location:** https://grouplens.org/datasets/movielens/

Import Libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Explorar os dados